In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

AttributeError: module 'torch' has no attribute 'device'

In [6]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [9]:

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)


NameError: name 'AutoTokenizer' is not defined

In [5]:
from datasets import load_dataset

dataset = load_dataset("csv", split='train', data_files="../dataset/data.csv")

In [6]:
def tokenize(examples):
    token = tokenizer(examples['clean_text'], truncation=True, padding="max_length")
    return token

tokenized_dataset = dataset.map(tokenize)

In [7]:

no = 10
text = dataset['clean_text'][no]
text = preprocess(text)
sentiment = dataset['sentiment'][no]

test_input = tokenizer(text, return_tensors='pt')
print(test_input)

all_encodings = []
for tweet in dataset['clean_text']:
    encoded_tweet = tokenizer(tweet, return_tensors='pt')
    all_encodings.append(encoded_tweet)


{'input_ids': tensor([[    0,   605, 41407,   127,  3822,    21,  1105,    65,   200,   536,
             8,   122,    16,  1132,   885, 41407,    16,    42,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [31]:
for i, encoded_tweet in enumerate(all_encodings[:10]):
    output = model(**encoded_tweet)
    # print(output.losses, end = '\t')
    score = output[0][0].detach().numpy()
    score = softmax(score)
    print(np.argmax(score) - 1, end='\t')
    print(dataset['clean_text'][i])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [9]:

scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]

for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")



1) positive 0.5212
2) neutral 0.4527
3) negative 0.0261


In [17]:
from transformers import TFTrainingArguments
training_args = TFTrainingArguments("my_model")
from transformers import Trainer
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset, tokenizer=tokenizer)

/mnt/Data/projects/roberta/lib/python3.11/site-packages/transformers/training_args_tf.py:293: FutureWarning: The n_gpu argument is deprecated and will be removed in a future version, use n_replicas instead.
  warnings.warn(


In [26]:
from transformers import AdamW
import torch
train_loader = torch.utils.data.DataLoader(
    tokenized_dataset,
    shuffle=True,
)


In [28]:

optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
import torch
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(batch['input_ids'][0])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

/mnt/Data/projects/roberta/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/33027 [00:55<?, ?it/s]


IndexError: too many indices for tensor of dimension 1